Name: Aditya Gupte

Institute: IIT Gandhinagar

Date: May 3, 2024

In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.classify import NaiveBayesClassifier

In [3]:
df_train = pd.read_csv('twitter_training.csv', encoding = 'latin-1')
df_train.describe()

,2401
count,74681.000000
mean,6432.640149
std,3740.423819
min,1.000000
25%,3195.000000
50%,6422.000000
75%,9601.000000
max,13200.000000


In [4]:
df_train.head()
#df_train.tail()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [5]:
df_train.iloc[:,2].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Aditya
[nltk_data]     Gupte\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [23]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Aditya
[nltk_data]     Gupte\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [6]:
# Preprocess the data
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

In [7]:
def preprocess_text(text):
    if isinstance(text, str):  # Check if the value is a string
        tokens = word_tokenize(text.lower())
        filtered_tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]
        return filtered_tokens
    else:
        return []  # Return an empty list for non-string values

In [28]:
# Extract features and labels

sentiment_mapping = {'Positive': 0, 'Neutral': 1, 'Negative': 2, 'Irrelevant': 1}

def dataset_format(data):
    all_texts = data.iloc[:, 3].values  # Assuming text data is in the 4th column
    labels = data.iloc[:, 2].values  # Assuming sentiment labels are in the 3rd column

    # Map sentiment labels to numeric values
    labels_numeric = [sentiment_mapping[label] for label in labels]

    feature_sets = [(dict([(token, True) for token in preprocess_text(text)]), label) for text, label in zip(all_texts, labels_numeric)]
    
    return feature_sets

In [26]:
# Split into training and testing sets
feature_sets = dataset_format(df_train)

train_size = int(0.8 * len(feature_sets))  # Use 80% for training
train_set = feature_sets[:train_size]
test_set = feature_sets[train_size:]

# Train the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(train_set)

In [27]:
# Evaluate accuracy on the testing set
test_accuracy = nltk.classify.accuracy(classifier, test_set)
print(f"Test Accuracy: {test_accuracy:.2f}")

Test Accuracy: 0.58


In [29]:
# Example usage
def get_senty(test_text):
    test_features = dict([(token, True) for token in preprocess_text(test_text)])
    sentiment = classifier.classify(test_features)

    # Map numeric sentiment back to labels
    sentiment_labels = {v: k for k, v in sentiment_mapping.items()}
    predicted_sentiment = sentiment_labels[sentiment]
    print(f"Predicted Sentiment: {predicted_sentiment}")

In [30]:
get_senty("awesome")

Predicted Sentiment: Positive


In [33]:
classifier2 = NaiveBayesClassifier.train(feature_sets)

In [31]:
data_predict = pd.read_csv('twitter_validation.csv', encoding='latin-1')
data_predict.head()

,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tomâs great auntie as âHayley canât get out of bedâ and told to his grandma, who now thinks Iâm a lazy, terrible person ð¤£"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp Iâve had Madeleine McCann in my c...


In [32]:
feature_pred = dataset_format(data_predict)

pred_accuracy = nltk.classify.accuracy(classifier, feature_pred)
print(f"Prediction dataset Accuracy: {pred_accuracy:.2f}")

Prediction dataset Accuracy: 0.79


In [34]:
pred_accuracy_2 = nltk.classify.accuracy(classifier2, feature_pred)
print(f"Prediction dataset Accuracy: {pred_accuracy_2:.2f}")

Prediction dataset Accuracy: 0.82
